<a href="https://colab.research.google.com/github/janbanot/msc-cs-code/blob/main/sem3/DL/DL_2025_Lab7-a.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Opcjonalna instalacja gymnasium

!uv pip install gymnasium
!uv pip install "gymnasium[toy-text]"

In [ ]:
import gymnasium as gym
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from IPython.display import HTML

def enable_video(env, interval=200):
    """
    Monkey-patch `env` so that:
      - env.frames will accumulate all rgb_array frames whenever you call env.render()
      - env.get_video() returns an IPython.display.HTML object of the animation.
    """
    if env.render_mode != 'rgb_array':
        env.get_video = lambda : "Render mode should be 'rgb_array' to get a video"
        return env  # No recording

    env.frames = []
    orig_render = env.render

    def _render_and_capture(*args, **kwargs):
        frame = orig_render(*args, **kwargs)  # get the RGB array
        env.frames.append(frame)
        return frame

    env.render = _render_and_capture

    def get_video():
        if not env.frames:
            return "No frames in env.frames; make sure you called env.render() at least once."
        fig, ax = plt.subplots(figsize=(3, 3), tight_layout=True)
        ax.axis("off")
        img = ax.imshow(env.frames[0], interpolation='none', animated=True)

        def _update(i):
            img.set_array(env.frames[i])
            return (img,)

        ani = FuncAnimation(fig, _update,
                            frames=len(env.frames), interval=interval,
                            blit=True, repeat=False)
        plt.close(fig)
        return HTML(ani.to_html5_video())

    env.get_video = get_video
    return env

In [ ]:
env = gym.make("FrozenLake-v1", map_name="4x4", is_slippery=False,
               render_mode='rgb_array')

enable_video(env)

state, info = env.reset()
terminated = False
truncated = False
while not (terminated or truncated):
    action = env.action_space.sample()  # Losowa akcja
    # Przejście do kolejnego stanu
    state, reward, terminated, truncated, info = env.step(action)
    print(f'{state = }')
    break

    if env.render_mode:
        env.render()   # Dodaje kolejną klatkę animacji

env.close()

env.get_video()

state = 0


'No frames in env.frames; make sure you called env.render() at least once.'

# Q-learning

In [ ]:
import numpy as np

def train(episodes):
    env = gym.make("FrozenLake-v1", map_name="4x4", is_slippery=False,
                render_mode=None)

    q = np.zeros((env.observation_space.n, env.action_space.n))
    print(f'{q.shape = }')

    learning_rate = 0.9
    discount_factor = 0.9

    epsilon = 1
    epsilon_decay_rate = 1 / episodes
    rng = np.random.default_rng()
    rewards = np.zeros(episodes)

    for i in range(episodes):
        state = env.reset()[0]
        terminated = False
        truncated = False

        while not (terminated or truncated):
            if rng.random() < epsilon:
                action = env.action_space.sample()
            else:
                action = np.argmax(q[state, :])

            new_state, reward, terminated, truncated, _ = env.step(action)

            # Aktualizacja funkcji (tablicy) stanu-akcji
            q[state, action] = q[state, action] + learning_rate * (
                reward + discount_factor * np.max(q[new_state, :]) - q[state, action]
            )
            rewards[i] += reward

            state = new_state

        epsilon = max(epsilon - epsilon_decay_rate, 0)
        learning_rate = max(learning_rate - 1 / episodes, 1e-4)

    env.close()

    sum_rewards = np.zeros(episodes)
    for t in range(episodes):
        sum_rewards[t] = np.sum(rewards[max(0, t-100) : t+1])

    plt.plot(sum_rewards)
    plt.xlabel('Episode')
    plt.ylabel('Reward (past 100 episodes)')
    plt.show()
    return q

q_table = train(15_000)

In [ ]:
env = gym.make("FrozenLake-v1", map_name="4x4", is_slippery=False,
               render_mode='rgb_array')

enable_video(env, interval=300)

state, info = env.reset()
terminated = False
truncated = False
while not (terminated or truncated):
    action = np.argmax(q_table[state, :])
    state, reward, terminated, truncated, info = env.step(action)
    env.render()

env.close()

env.get_video()

# Zad. 1.

Sprawdź działanie algorytmu Q-learning dla problemu FrozenLake-v1 i mapy "8x8".
* Rozważ wariant `is_slippery = False` oraz `is_slippery = True`
* Uruchom algorytm kilka razy, czy zawsze otrzymujemy zbieżność?